In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "data_prep", ENTITY_NAME: "train_data"},
      { STEP_NAME: "data_prep", ENTITY_NAME: "eval_data"},
      { STEP_NAME: "data_prep", ENTITY_NAME: "train_eval_config"}
    ],
    
    tmp_outputs =
    [
        { ENTITY_NAME: "load_train_data" },
        { ENTITY_NAME: "load_eval_data" },
        { ENTITY_NAME: "aug_dataset" },
        { ENTITY_NAME: "train_eval_config" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 0_cache_datasets](./imgs/0_cache_datasets.drawio.png)

In [ ]:
#4 get substep.interface
inputs = substep.inputs(step_name = "data_prep")
outputs = substep.outputs()
tmp_outputs = substep.tmp_outputs()


In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Loading train and val datasets and configuration files (from the previous component data_prep)

In [ ]:
import os.path as osp
import os
from sinara.store import SinaraStore
from pathlib import Path

# copy config from previos step to outputs
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.train_data, tmp_dir=tmp_outputs.load_train_data)
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.eval_data, tmp_dir=tmp_outputs.load_eval_data)
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.train_eval_config, tmp_dir=tmp_outputs.train_eval_config)

#### Unpack parquet from load_train_data and load_eval_data by files to aug_dataset

In [ ]:
from functools import partial

### Save dataset from parquet to files
def save_file(row, tmp_dir): 
    total_img_path = osp.join(tmp_dir, row.file_names)
    os.makedirs(osp.dirname(total_img_path), exist_ok=True)
    with open(total_img_path, 'wb') as f_id:
        f_id.write(row.files_binary)

In [ ]:
%%time

# LOAD Train Images

print(f"spark read start")
df_spark = spark.read.parquet(tmp_outputs.load_train_data)
df_spark.foreach(partial(save_file, tmp_dir=tmp_outputs.aug_dataset))

In [ ]:
%%time

# LOAD Valid Images

print(f"spark read start")
df_spark = spark.read.parquet(tmp_outputs.load_eval_data)
df_spark.foreach(partial(save_file, tmp_dir=tmp_outputs.aug_dataset))

In [ ]:
#9 stop spark
SinaraSpark.stop_session()